# Item Genres Scrapping

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import sys
sys.path.append('../../src')

from domain_context import DomainContext

import time
import util as ut

### Setup

In [14]:
ctx = DomainContext(host='http://localhost:8000')

In [15]:
items_df = ctx.item_service.find_all()

2023-01-02 20:19:02,993 :: INFO :: ItemRepository :: Page 1/3 - Items 500/1029
2023-01-02 20:19:03,228 :: INFO :: ItemRepository :: Page 2/3 - Items 1000/1029
2023-01-02 20:19:03,291 :: INFO :: ItemRepository :: Page 3/3 - Items 1029/1029
2023-01-02 20:19:03,291 :: INFO :: ItemRepository :: 1029 Total Items 


In [69]:
remove = ["Folio Society Hardcover",
    "Performed by Derek Jacobi, Geraldine McEwan &amp; Cast",
    'How a Senior Devil Instructs a Junior Devil in the Art of Temptation',
    'with related readings',
    'Individual Reader Narrative of the Life of Frederick Douglas',
    '(HRW Library Study Guides)',
    'Individual Reader 1984',
    'The New Translation (Harper Perennial Modern Classics)',
    "(Contemporary Classics)",
    'With Connected Readings',
    '(Contemporary American Fiction)',
    '(Classic, 20th-Century, Audio)',
    '(An Isaac Bell Adventure)',
    "S. Morgenstern's Classic Tale of True Love and High Adventure",
    '(Spanish language) (Spanish Edition)',
    "(Movie Tie-in Edition): A No. 1 Ladies' Detective Agency Novel (1)",
    '(Marissa Meyer)',
    '(The Jake Brigance)',
    '(Virago Modern Classics) [Paperback] [Jul 16, 2015] Daphne Du Maurier (VMC,VMC Designer Collection)',
    '(Dresden Files (Unnumbered Paperback))',
    '(The Oklahoma All-Girl Brands) (Silhouette Intimate Moments, 1039)',
    '(Random House Large Print)',
    '(Sookie Stackhouse/True Blood, Book 1)',
    'The heartbreaking Richard and Judy bestseller',
    '(Kingkiller Chronicles, Day 1)',
    '(Turtleback School &amp; Library Binding Edition) (Hcc)',
    '(Turtleback School &amp; Library Binding Edition) (Cedar Ridge)',
    '(The McCarthys of Gansett Island Series) (Volume 1)',
    '(The Texas Brand) (Silhouette Intimate Moments, No 716)',
    '(The Mitch Rapp Series)',
    '(The Woodlands)',
    '(The Driven Trilogy)',
    '(A Standalone Romantic Comedy) (The Pucked Series',
    '(The Starcrossed Series)',
    '(A Sunset Cove Novel)',
    '(Throne of Glass)',
    '(Thorndike Press Large Print Core)',
    '(Thorndike Press Large Pring Basic)',
    "(Ellora's Cave Moderne)",
    "The year's best suspense novel",
    '(Amos Decker series)',
    '(A This Man Novel)',
    '(A Rose Gardner Mystery)',
    '(Tempting Too Far Novel)',
    '(The REAL series)',
    '(The REAL series)',
    '(The Tangled Series)',
    '(The REAL series)',
    '(Sign of Love)',
    ': Paranormal BBW Romance (Grayslake)',
    '(Seductive Nights: Julia &amp; Clay Book 1)',
    '(The Nine Minutes Trilogy)',
    '(A Knights Rebels MC Book 1)',
    '(BBW Paranormal Shape Shifter Romance): A BBW in need of a date + Two hot Alphas looking for a mate = The hottest triad ever. (Paranormal Dating Agency) (Volume 1)',
    ': Inside Out (Careless Whispers)',
    ': Inside Out (Careless Whispers)',
    '(The Hudson Valley Series)',
    '(The Coming Home Series)',
    '(The Chicago Rebels Series)',
    '(The Chicago Rebels Series)',
    '(The Coming Home Series)',
    '(A Sign of Love)',
    '(A Jewel Novel)',
    '(Sign of Love)',
    'King Part',
    '(The Nine Minutes Trilogy)',
    'A Series of Standalone Novels',
    '(Cocky Bastard)',
    '(Skylar and Sebastian) (Happy Crazy Love)',
    'King Series',
    '(Hearts in Darkness Duet)',
    '(The Manwhore Series)',
    'a survival romance novel',
    '(Bad Boy Billionaires)',
    '(Stand-alone British Romance) (British Bad Boys)',
    '(The Stroked Series)',
    '(A Twin Estates Novel)',
    '(Anti-Heroes Collection',
    '(A Twin Estates Novel)',
    'Flow, The Grip Prequel',
    'Co-WRECKER',
    '(The Elements Series)',
    'A Standalone Off-Limits Romance',
    '(The Pucked Series)',
    '(Tight Spaces)',
    'Preppy: The Life and Death of Samuel Clearwater 2',
    "(Heroes of Chance Creek)",
    "(Cowboys of Chance Creek)",
    "Not Everything Should Come in 3's",
    "The Complete Collection: Mine For Tonight, Mine For Now, Mine Forever, Mine Completely",
    '(The Kingwood Duet)',
    '(The Wicked Horse Series)',
    '(Last Call Series)',
    '(The Everyday Heroes Series)',
    '(The Everyday Heroes)',
    '(The Winston Brothers)',
    '(The Anti-Heroes Collection)',
    'An Anti-Heroes Collection Novel (The Savage Trilogy)',
    '(The Darkest Sunrise Duet)',
    'A North Ridge Novel',
    'Miss Fix-It',
    '(British Bad Boys)',
    'Until Her/ Until Him',
    'A King Series Novel',
    '(Spanish Edition)'
    'Correr o Morir', 
    'A London Legends Christmas Romance',
    '(King Series)',
    '(The Syndicate Series)',
    "the Covenant/Autobiography of Will Rogers/Cry, the Beloved Country (Reader's Digest Condensed Books, Volume 1: 1950)",
    '(Scholastic Classics)',
    'Maus 1 and 2',
    '(Word Cloud Classics)',
    '(Lord of the Rings Part 1)',
    'By Charles Dickens - The Franklin Library (Hardcover - 1979)',
    'The Japanese Art of Decluttering and Organizing',
    '[ Lies BY Grant, Michael ( Author ) ] { Paperback } 2014',
    'Kindle edition'
]

In [86]:
from selectorlib import Extractor
import requests 
import re


class GoodReadsScrapper:    
    def __init__(self):
        self.remove_strings = [
            'A Sugar Bowl Novel',
            '/True',
            'Trilogy',
            'Novel',
            'A Harry Bosch',
            'A Reacher',
            ','
        ] + remove

        self.remove_after = [
            'A Novel',
            'A Jack Reacher Novel',
            'Collins',
            'Modern Classics',
            'Kindle',
            'by',
            'By',
            'Penguin',
            '20thCentury',
            ' A ',
            ' An ',
            'English Edition',
            'Series',
            'Hardcover',
            'Large Print',
            'Anniversary Edition',
            'Motorcycle Club',
            'Marissa Meyer'
        ]
    
    def by_name(self, name, id):
        for remove_string in self.remove_strings:
            name = name.replace(remove_string, '').strip()

        for after in self.remove_after:
            name = name.split(after)[0].strip()

        name = name.replace(":",' ').replace("'",'').replace('-', '').replace('(', '').replace(')', '').replace('&amp;', '').replace('.', '')
        name = re.sub(' +', ' ', name).strip()
        name = name.replace(' ', '+')
        name = name.replace('Volume', '')
        name = name.replace('Vol', '')
        name = name.replace('Book', '')
        


        print(f'Search: {id} {name}')
        r = requests.get(f'https://www.goodreads.com/search?q={name}')
        e = Extractor.from_yaml_string("""
        items:
            css: tr[itemtype="http://schema.org/Book"]
            multiple: true
            type: Link
        """)
        items = e.extract(r.text)['items']
        if items == None:
            print(f'Not found results...')
            return []
        else:
            return [f'https://www.goodreads.com/{i}'for i in items]

    def genres(self, link):
        r = requests.get(link)
        e = Extractor.from_yaml_string("""
        genres:
            css: span.BookPageMetadataSection__genreButton
            multiple: true
            type: Text
        """)
        return e.extract(r.text)['genres']
    
    def genres2(self, link):
        r = requests.get(link)
        e = Extractor.from_yaml_string("""
        genres:
            css: a.bookPageGenreLink
            multiple: true
            type: Text
        """)
        return e.extract(r.text)['genres']

In [87]:
scrapper = GoodReadsScrapper()

try:
    books = ut.Picket.load('./books_genres')
except:
    books = {}

In [89]:
for index, row in items_df.iterrows():
    name = row["name"]

    if name in books and len(books[name]) > 0:
        # print(f'Success: {name}')
        continue 

    items = scrapper.by_name(name, row["id"])
    if len(items) == 0:
        books[name] = []
        ut.Picket.save('./books_genres', books)
        continue
    
    url = items[0]
    # print(f'Url: {url}')
    time.sleep(1)
    genres = scrapper.genres(url)
    
    if genres == None:
        genres = scrapper.genres2(url)
        time.sleep(1)
        if genres == None:    
            print(f'Not found genres...')
            books[name] = []
            ut.Picket.save('./books_genres', books)
            continue

    books[name] = genres
    print(f'Success: {name}: {genres}')
    ut.Picket.save('./books_genres', books)

Search: 2095 Holt+McDougal+Library+High+School+with+Connections

Not found genres...

Search: 2097 Holt+McDougal+Library+High+School+with+Connections

Not found genres...

Search: 19911 The+Princess+Bride+Fox

...

Not found results...

Search: 215284 
The+Show+Must+Go+On/The+Cry+and+the+Covenant/Autobiography+of+Will+Rogers/Cry+the+Beloved+Country+Readers+Digest+Co
ndensed+s++1+1950

Not found results...

Search: 223343 [+Lies+BY+Grant+Michael+Author+]+{+Paperback+}+2014

Not found results...

In [ ]:
[key  for key, value in books.items() if len(value) == 0]

In [ ]:
books

In [90]:
sum([1 if len(value) == 0 else 0 for key, value in books.items()])

22

In [ ]:
sum([1 for key, value in books.items() if len(value) == 0])

In [ ]:
del books['title']

In [ ]:
[key for key, value in books.items() if len(value) == 0 and 'Book' in key ]